In [1]:
from pyccx import *
import numpy as np
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor, ConstantUnderRelaxation
from mapper import generate_mapper
from add_pyofm import pyofm_path
import sys
sys.path.append(pyofm_path)
from mpi4py import MPI
import pyofm as ofm
import time

In [2]:
ccx_logoff = True
if ccx_logoff:
    import logging
    logger = logging.getLogger('PYCCX')
    logger.setLevel(logging.CRITICAL)

use_relax = True
under_relax_obj = DynamicUnderRelaxation
init_omega = 0.95
fluid_dup_copy = 3  # for each side

# tolerance setting
tol = 1e-3
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
tpoint = time.time()
solverF = ofm.make_solver('buoyantPimpleFoam' ,kase='fluid')
solverF.create_fields()
solverF.initialize()
solverF.enable_backup_restore()
dt = solverF.init_dt()
iface = ofm.iface.InterfaceAdapter(solverF, 'interface')
fnodes = iface.extract_centres()
ftempAdap = ofm.CHT.TemperatureAdapter(iface)
ffluxAdap = ofm.CHT.HeatFluxAdapter(iface)
fluid_prepare_time = time.time()-tpoint
print(fluid_prepare_time)

0.1529710292816162


In [4]:
# solid solver
tpoint = time.time()
jobname = 'solid/solidNbi1'
output = 'asc'
prob = Problem(jobname=jobname, output=output)
msolver = prob.get_solver()
msolver.initailize()
# since we know there is one step
# we stop b4 it
msolver.solve(stopb4step=1)
nlgeom = msolver.get_nlgeomsolver()
nlgeom.initialize()

# initialize our interface set
iset = nlgeom.get_set(
    set_name='Interface',
    set_type='sfelem',
    is_surface=True
)
nset = nlgeom.get_set(
    set_name='Inodes',
    set_type='node',
    is_surface=True
)

# retrieve geometry data
snodes = nset.coordinates()
scents = iset.extract_face_centers()

# create cht adapter
solverS = CHTAdapter(nlgeom=nlgeom)
solverS.add_interface(set_acc=nset, is_settable=False, itype=CHTAdapter.TEMP)
solverS.add_interface(set_acc=iset, is_settable=True, itype=CHTAdapter.DFLUX)

# now we are ready
solid_prepare_time = time.time()-tpoint
print(solid_prepare_time)

0.014734506607055664


In [5]:
def dup_fluid_mesh(fnodes, dist, copy):
    """This is for duplicating the fluid mesh in z-direction for supporting interpolation"""
    n = fnodes.shape[0]
    dist_ = fnodes[0, 2] #  input original z position
    temp = fnodes.copy()
    for i in range(2*copy):
        temp = np.concatenate((temp, fnodes), axis=0)
    for i in range(copy):
        temp[(i+1)*n:(i+2)*n, 2] = dist_+(i+1)*dist
        temp[(copy+i+1)*n:(copy+i+2)*n, 2] = dist_-(i+1)*dist
    return n, temp

def dup_fluid_solu(solu, copy):
    """assume scalar field"""
    n = solu.size
    my_solu = np.empty((2*copy+1)*n, dtype=float)
    for i in range(2*copy+1):
        my_solu[i*n:(i+1)*n]=solu
    return my_solu

In [6]:
(freal_size, fnodes) = dup_fluid_mesh(fnodes, 0.002, fluid_dup_copy)

In [7]:
tpoint = time.time()
Tf = 'Tf'
Ts = 'Ts'
Ff = 'Ff'
Fs = 'Fs'
Ff2Fs = {Ff:Fs}
Ts2Tf = {Ts:Tf}
f2s_mapper, s2f_mapper = generate_mapper(fnodes, [scents, snodes], Ff2Fs, Ts2Tf,r1=0.02,r2=0.02)
mapper_construct_time = time.time()-tpoint
print(mapper_construct_time)

0.42379117012023926


In [8]:
# coupling data and setups
fluxF_dup = InterfaceData(size=fnodes.shape[0], value=0.0)
fluxF = InterfaceData(size=freal_size, value=0.0)
fluxS = InterfaceData(size=scents.shape[0], value=0.0)
tempF_dup = InterfaceData(size=fnodes.shape[0], value=1000.0)
tempF = InterfaceData(size=freal_size, value=1000.0)
tempS = InterfaceData(size=snodes.shape[0], value=800.0) # initial guess
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = under_relax_obj(init_omega=init_omega)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [9]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
tF = 10.0
DT = 10.0*dt
avg_pc_iterations = 0.0
flog = open('FDSN.log', mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting\n')
flog.close()
flog = open('FDSN.log', mode='a')
flog.write(
    '\nFluid initialization time: %f\nSolid initialization time: %f\nMapper constuction time: %f\n\n' % \
    (fluid_prepare_time, solid_prepare_time, mapper_construct_time)
)

108

In [10]:
t = 0.0
step = 0

In [11]:
def tol(idata):
    return np.linalg.norm(idata.res)/np.linalg.norm(idata.curr)

In [12]:
tpoint = time.time()
# begin to couple
while t <= tF+1e-6:
    t += DT
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_state()
    solverS.backup_state()
    
    # mapping, T:s->f
    s2f_mapper.put(Ts, tempS.curr)
    s2f_mapper.apply(Ts, Tf)
    tempF_dup.curr[:] = s2f_mapper.get(Tf)
    tempF.curr[:] = tempF_dup.curr[0:freal_size]
    
    # update fluid interface temperature
    ftempAdap.assign(tempF.curr)
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.subcycle(DT)
        # retrieve fluid interface flux
        fluxF.curr[:] = ffluxAdap.extract()
        
        fluxF_dup.curr[:] = dup_fluid_solu(fluxF.curr, fluid_dup_copy)
        
        # mapping, F:f->s
        f2s_mapper.put(Ff, fluxF_dup.curr)
        f2s_mapper.apply(Ff, Fs)
        fluxS.curr[:] = f2s_mapper.get(Fs)
        # update the distributed heat fluxes on solid interface
        solverS['Interface', SET].set_dfluxes(fluxS.curr)
        
        # advance solid
        solverS.adjust_timesize(DT)
        solverS.increment()
        tempS.curr[:] = solverS['Inodes', GET].get_temperatures()
        
        # mapping, T:s->f
        s2f_mapper.put(Ts, tempS.curr)
        s2f_mapper.apply(Ts, Tf)
        tempF_dup.curr[:] = s2f_mapper.get(Tf)
        
        tempF.curr[:] = tempF_dup.curr[0:freal_size]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            # notice the solid side to advance
            solverS.finish_increment()
            solverF.advance()
            dt = solverF.dt()
            DT = 10.0*dt
            if t + DT > tF:
                DT = tf-t
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF, step, pc_counts)
                under_relax.update_solution(tempF)
            # update the underelaxed interface temperature
            # make sure that restoring the fluid solver will
            # not affect the newly updated values
            solverF.restore_state()
            ftempAdap.assign(tempF.curr)
            solverS.restore_state()
            solverS.finish_increment()
            pc_counts = pc_counts + 1
            if pc_counts > 20 and pc_counts % 5 == 0:
                print('WARNING too slow convergence for step %d with correction iterations %d, current relative error %e..' % (step+1, pc_counts, tol(tempF)))

    # outputs
    step += 1
    msg = 'step=%d: res=%e, coupling_dt=%f, time=%f, pc_iterations=%i.' % (step, tol(tempF), DT, t, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts

nlgeom.finalize()
msolver.solve(skipsolve=True)
msolver.finalize()

solving_time=  time.time()-tpoint
flog.write('\nSolving time: %f\n' % solving_time)
flog.close()

step=1: res=6.842476e-05, coupling_dt=0.000206, time=0.000000, pc_iterations=3.
step=2: res=6.137247e-04, coupling_dt=0.000023, time=0.000207, pc_iterations=10.
step=3: res=8.887900e-04, coupling_dt=0.000028, time=0.000230, pc_iterations=2.
step=4: res=7.048452e-04, coupling_dt=0.000062, time=0.000257, pc_iterations=3.
step=5: res=9.063802e-04, coupling_dt=0.000104, time=0.000320, pc_iterations=4.
step=6: res=7.851737e-04, coupling_dt=0.000186, time=0.000424, pc_iterations=5.
step=7: res=5.710705e-04, coupling_dt=0.000186, time=0.000610, pc_iterations=4.
step=8: res=6.067157e-04, coupling_dt=0.000185, time=0.000796, pc_iterations=3.
step=9: res=4.648523e-04, coupling_dt=0.000179, time=0.000981, pc_iterations=3.
step=10: res=3.778653e-04, coupling_dt=0.000117, time=0.001161, pc_iterations=4.
step=11: res=3.171871e-04, coupling_dt=0.000054, time=0.001278, pc_iterations=3.
step=12: res=5.464920e-04, coupling_dt=0.000166, time=0.001332, pc_iterations=3.
step=13: res=6.974038e-04, coupling_

step=103: res=4.495532e-04, coupling_dt=0.000187, time=0.011176, pc_iterations=3.
step=104: res=7.886952e-04, coupling_dt=0.000187, time=0.011363, pc_iterations=2.
step=105: res=6.268216e-04, coupling_dt=0.000186, time=0.011551, pc_iterations=3.
step=106: res=5.278313e-04, coupling_dt=0.000172, time=0.011737, pc_iterations=3.
step=107: res=4.298392e-04, coupling_dt=0.000181, time=0.011909, pc_iterations=3.
step=108: res=7.884012e-04, coupling_dt=0.000085, time=0.012090, pc_iterations=2.
step=109: res=6.413952e-04, coupling_dt=0.000031, time=0.012175, pc_iterations=3.
step=110: res=2.894803e-04, coupling_dt=0.000037, time=0.012206, pc_iterations=2.
step=111: res=3.553844e-04, coupling_dt=0.000045, time=0.012243, pc_iterations=2.
step=112: res=5.891155e-04, coupling_dt=0.000030, time=0.012288, pc_iterations=2.
step=113: res=9.859637e-04, coupling_dt=0.000036, time=0.012318, pc_iterations=2.
step=114: res=9.322691e-04, coupling_dt=0.000080, time=0.012354, pc_iterations=3.
step=115: res=9.

step=203: res=7.226022e-04, coupling_dt=0.000186, time=0.023256, pc_iterations=3.
step=204: res=6.405242e-04, coupling_dt=0.000171, time=0.023442, pc_iterations=3.
step=205: res=9.284163e-04, coupling_dt=0.000039, time=0.023613, pc_iterations=2.
step=206: res=8.203101e-04, coupling_dt=0.000087, time=0.023651, pc_iterations=3.
step=207: res=4.505087e-04, coupling_dt=0.000188, time=0.023738, pc_iterations=2.
step=208: res=4.900965e-04, coupling_dt=0.000188, time=0.023926, pc_iterations=2.
step=209: res=7.213642e-04, coupling_dt=0.000188, time=0.024113, pc_iterations=2.
step=210: res=6.408255e-04, coupling_dt=0.000188, time=0.024301, pc_iterations=3.
step=211: res=9.605122e-04, coupling_dt=0.000188, time=0.024488, pc_iterations=2.
step=212: res=8.228919e-04, coupling_dt=0.000188, time=0.024676, pc_iterations=3.
step=213: res=6.407165e-04, coupling_dt=0.000187, time=0.024864, pc_iterations=3.
step=214: res=9.289688e-04, coupling_dt=0.000186, time=0.025051, pc_iterations=2.
step=215: res=8.

step=303: res=8.583835e-04, coupling_dt=0.000037, time=0.035027, pc_iterations=3.
step=304: res=4.038618e-04, coupling_dt=0.000044, time=0.035063, pc_iterations=2.
step=305: res=4.326784e-04, coupling_dt=0.000035, time=0.035108, pc_iterations=2.
step=306: res=6.296918e-04, coupling_dt=0.000042, time=0.035142, pc_iterations=2.
step=307: res=5.929561e-04, coupling_dt=0.000093, time=0.035184, pc_iterations=3.
step=308: res=7.002668e-04, coupling_dt=0.000182, time=0.035277, pc_iterations=2.
step=309: res=8.455524e-04, coupling_dt=0.000131, time=0.035459, pc_iterations=2.
step=310: res=6.240769e-04, coupling_dt=0.000076, time=0.035590, pc_iterations=3.
step=311: res=9.731528e-04, coupling_dt=0.000154, time=0.035666, pc_iterations=2.
step=312: res=6.467173e-04, coupling_dt=0.000044, time=0.035820, pc_iterations=3.
step=313: res=5.620404e-04, coupling_dt=0.000092, time=0.035864, pc_iterations=3.
step=314: res=4.581890e-04, coupling_dt=0.000171, time=0.035956, pc_iterations=2.
step=315: res=3.

step=403: res=5.501647e-04, coupling_dt=0.000188, time=0.046690, pc_iterations=2.
step=404: res=5.177359e-04, coupling_dt=0.000188, time=0.046877, pc_iterations=2.
step=405: res=5.836599e-04, coupling_dt=0.000188, time=0.047065, pc_iterations=2.
step=406: res=6.729112e-04, coupling_dt=0.000188, time=0.047253, pc_iterations=2.
step=407: res=7.745441e-04, coupling_dt=0.000188, time=0.047440, pc_iterations=2.
step=408: res=8.795423e-04, coupling_dt=0.000188, time=0.047628, pc_iterations=2.
step=409: res=6.581652e-04, coupling_dt=0.000188, time=0.047815, pc_iterations=3.
step=410: res=1.954475e-04, coupling_dt=0.000187, time=0.048003, pc_iterations=2.
step=411: res=1.629211e-04, coupling_dt=0.000186, time=0.048190, pc_iterations=2.
step=412: res=7.265798e-04, coupling_dt=0.000167, time=0.048376, pc_iterations=0.
step=413: res=7.901735e-04, coupling_dt=0.000168, time=0.048543, pc_iterations=2.
step=414: res=8.322885e-04, coupling_dt=0.000179, time=0.048711, pc_iterations=2.
step=415: res=9.

...

step=2589: res=8.600959e-04, coupling_dt=0.000033, time=0.294210, pc_iterations=3.
step=2590: res=6.503436e-04, coupling_dt=0.000075, time=0.294244, pc_iterations=3.
step=2591: res=9.767492e-04, coupling_dt=0.000149, time=0.294319, pc_iterations=2.
step=2592: res=8.742234e-04, coupling_dt=0.000175, time=0.294468, pc_iterations=3.
step=2593: res=4.585505e-04, coupling_dt=0.000027, time=0.294643, pc_iterations=2.
step=2594: res=2.574802e-04, coupling_dt=0.000032, time=0.294669, pc_iterations=2.
step=2595: res=2.782850e-04, coupling_dt=0.000039, time=0.294702, pc_iterations=2.
step=2596: res=5.113411e-04, coupling_dt=0.000047, time=0.294741, pc_iterations=2.
step=2597: res=5.399085e-04, coupling_dt=0.000094, time=0.294787, pc_iterations=3.
step=2598: res=7.787051e-04, coupling_dt=0.000188, time=0.294881, pc_iterations=2.
step=2599: res=8.804354e-04, coupling_dt=0.000188, time=0.295069, pc_iterations=2.
step=2600: res=9.434388e-04, coupling_dt=0.000187, time=0.295256, pc_iterations=2.
step

step=2688: res=6.478959e-04, coupling_dt=0.000041, time=0.306787, pc_iterations=2.
step=2689: res=6.909738e-04, coupling_dt=0.000092, time=0.306828, pc_iterations=3.
step=2690: res=6.920833e-04, coupling_dt=0.000171, time=0.306921, pc_iterations=2.
step=2691: res=9.154419e-04, coupling_dt=0.000059, time=0.307091, pc_iterations=3.
step=2692: res=8.392821e-04, coupling_dt=0.000075, time=0.307150, pc_iterations=2.
step=2693: res=8.918674e-04, coupling_dt=0.000023, time=0.307225, pc_iterations=3.
step=2694: res=5.322979e-04, coupling_dt=0.000027, time=0.307248, pc_iterations=2.
step=2695: res=8.911818e-04, coupling_dt=0.000033, time=0.307275, pc_iterations=2.
step=2696: res=7.980846e-04, coupling_dt=0.000074, time=0.307308, pc_iterations=3.
step=2697: res=7.351718e-04, coupling_dt=0.000033, time=0.307382, pc_iterations=3.
step=2698: res=6.924822e-04, coupling_dt=0.000073, time=0.307414, pc_iterations=3.
step=2699: res=6.440497e-04, coupling_dt=0.000045, time=0.307488, pc_iterations=3.
step

step=2787: res=8.934278e-04, coupling_dt=0.000188, time=0.315652, pc_iterations=2.
step=2788: res=6.029603e-04, coupling_dt=0.000187, time=0.315839, pc_iterations=3.
step=2789: res=9.055495e-04, coupling_dt=0.000184, time=0.316027, pc_iterations=2.
step=2790: res=6.348979e-04, coupling_dt=0.000169, time=0.316211, pc_iterations=3.
step=2791: res=9.125747e-04, coupling_dt=0.000188, time=0.316380, pc_iterations=2.
step=2792: res=5.837751e-04, coupling_dt=0.000188, time=0.316567, pc_iterations=3.
step=2793: res=8.720619e-04, coupling_dt=0.000188, time=0.316755, pc_iterations=2.
step=2794: res=5.995606e-04, coupling_dt=0.000187, time=0.316942, pc_iterations=3.
step=2795: res=9.104932e-04, coupling_dt=0.000185, time=0.317130, pc_iterations=2.
step=2796: res=6.029239e-04, coupling_dt=0.000169, time=0.317314, pc_iterations=3.
step=2797: res=8.369813e-04, coupling_dt=0.000188, time=0.317483, pc_iterations=2.
step=2798: res=8.912565e-04, coupling_dt=0.000188, time=0.317671, pc_iterations=2.
step

step=2886: res=8.880520e-04, coupling_dt=0.000071, time=0.326319, pc_iterations=3.
step=2887: res=5.538967e-04, coupling_dt=0.000087, time=0.326390, pc_iterations=2.
step=2888: res=8.289750e-04, coupling_dt=0.000182, time=0.326478, pc_iterations=2.
step=2889: res=5.745234e-04, coupling_dt=0.000184, time=0.326660, pc_iterations=3.
step=2890: res=8.145848e-04, coupling_dt=0.000151, time=0.326844, pc_iterations=2.
step=2891: res=5.324927e-04, coupling_dt=0.000011, time=0.326995, pc_iterations=3.
step=2892: res=5.891020e-04, coupling_dt=0.000017, time=0.327006, pc_iterations=2.
step=2893: res=7.671884e-04, coupling_dt=0.000027, time=0.327023, pc_iterations=2.
step=2894: res=7.138463e-04, coupling_dt=0.000060, time=0.327050, pc_iterations=3.
step=2895: res=9.258937e-04, coupling_dt=0.000082, time=0.327109, pc_iterations=2.
step=2896: res=1.510530e-04, coupling_dt=0.000082, time=0.327191, pc_iterations=4.
step=2897: res=5.818424e-04, coupling_dt=0.000108, time=0.327273, pc_iterations=1.
step

KeyboardInterrupt: 